In [80]:
import RegionTrees
using Plots
gr()

Plots.GRBackend()

In [94]:
reload("RegionTrees")

In [207]:
module mpc

using JuMP
using Gurobi
using StaticArrays
using Interpolations
import RegionTrees

function run_mpc(q0, v0)
    model = Model(solver=GurobiSolver(OutputFlag=0))
    num_time_steps = 10
    dt = 0.1
    u_limit = 3
    C_q = 100
    c_vfinal = 100
    C_u = 1

    @variable model q[1:num_time_steps]
    @variable model v[1:num_time_steps]
    @variable model u[1:num_time_steps]

    @constraint model [i=2:num_time_steps] q[i] == q[i-1] + v[i-1] * dt
    @constraint model [i=2:num_time_steps] v[i] == v[i-1] + u[i-1] * dt
    @constraint model u .<= u_limit
    @constraint model u .>= -u_limit
    @constraint model q[1] == q0
    @constraint model v[1] == v0

    @objective model Min C_q * sum{q[i]^2, i=1:num_time_steps} + c_vfinal * v[end]^2 + C_u * sum{u[i]^2, i=1:num_time_steps}
    solve(model)

    getvalue(q), getvalue(v), getvalue(u)
end
# t = 0:dt:(dt * (num_time_steps - 1))

immutable MPCRefinery <: RegionTrees.AbstractRefinery
end

function evaluate(cell, point)
    p = (point - cell.boundary.origin) ./ cell.boundary.widths
    cell.data[p[1] + 1, p[2] + 1]
end

function RegionTrees.needs_refinement(::MPCRefinery, cell)
    x = RegionTrees.center(cell)
    value_interp = evaluate(cell, x)
    value_true = run_mpc(x[1], x[2])[3]
    !isapprox(value_interp, value_true, rtol=1e-1, atol=1e-1)
end

function RegionTrees.get_data(::MPCRefinery, boundary)
    f = v -> run_mpc(v[1], v[2])[3]
    interpolate(f.(RegionTrees.vertices(boundary)),
                BSpline(Linear()), OnGrid())
end

Base.one{T, N}(::Type{Array{T, N}}) = one(T)

end

mpc

In [208]:
q, v, u = mpc.run_mpc(1, 0)

([1.0,1.0,0.97,0.91,0.82,0.700112,0.574422,0.460014,0.368229,0.305808],[-0.0,-0.3,-0.6,-0.9,-1.19888,-1.2569,-1.14408,-0.917848,-0.624214,-0.324214],[-3.0,-3.0,-3.0,-2.98881,-0.580223,1.12825,2.2623,2.93634,3.0,-0.0])

In [209]:
function simulate(controller, q0, v0, dt, timespan)
    num_time_steps = timespan / dt
    qs = [q0]
    vs = [v0]
    ts = [0.0]
    q = q0
    v = v0
    for t in 0:dt:(timespan)
        u = controller(t, q, v)
        q += v * dt
        v += u * dt
        push!(qs, q)
        push!(vs, v)
        push!(ts, t)
    end
    ts, qs, vs
end
        

simulate (generic function with 1 method)

In [210]:
controller = (t, q, v) -> mpc.run_mpc(q, v)[3][1]

(::#39) (generic function with 1 method)

In [211]:
t, q, v = simulate(controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  8.59724e-5,5.36258e-5,2.83979e-5,9.84092e-6,-2.88847e-6,-1.08171e-5,-1.50039e-5,-1.64406e-5,-1.59925e-5,-1.43722e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000323466,-0.000252279,-0.00018557,-0.000127294,-7.9286e-5,-4.18688e-5,-1.43665e-5,4.48084e-6,1.6203e-5,2.23757e-5])

In [212]:
plot(q, v, xlim=(-10, 10), ylim=(-10, 10))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,192.476 596.063,198.132 595.221,203.788 593.536,209.444 591.009,215.101 587.64,220.757 583.429,226.413 578.375,232.069 572.479,237.725 565.741,243.381 
 558.16,249.038 549.737,254.694 540.472,260.35 530.365,266.006 519.415,271.662 507.623,277.318 494.989,282.975 481.512,288.631 467.194,294.287 452.033,299.943 
 436.029,305.599 419.183,301.483 402.951,295.827 387.56,290.171 373.012,284.515 359.306,278.859 346.443,273.202 334.421,267.546 323.242,261.89 312.906,256.234 
 303.411,250.578 294.759,244.921 286.949,239.265 279.982,233.609 273.856,227.953 268.573,222.297 264.132,216.641 260.534,210.984 257.778,205.328 255.864,199.672 
 254.792,194.016 254.563,188.36 255.176,182.704 256.631,177.047 258.929,171.391 262.068,165.735 266.05,160.079 270.875,154.423 276.542,148.767 283.05,146.511 
 289.895,148.899 296.384,154.328 302.065,159.984 306.904,165.64 310.9,171.296 314.054,176.953 316.365,182.609 317.835,187.561 318.567,191.149 318.764,193.562 
 318.603,195.02 318.224,195.744 317.737,195.936 317.222,195.771 316.731,195.391 316.297,194.903 315.936,194.389 315.651,193.899 315.439,193.466 315.292,193.105 
 315.198,192.822 315.146,192.611 315.126,192.464 315.128,192.371 315.144,192.32 315.167,192.3 315.193,192.302 315.219,192.318 315.242,192.341 315.262,192.367 
 315.279,192.393 315.291,192.417 315.3,192.437 315.305,192.453 315.309,192.465 315.311,192.474 315.311,192.48 315.31,192.483 315.309,192.485 315.308,192.485 
 315.307,192.484 315.305,192.483 315.304,192.482 315.303,192.481 315.303,192.479 315.302,192.478 315.302,192.477 315.302,192.477 315.301,192.476 315.301,192.476 
 315.301,192.476 315.301,192.476 
 "/>
 
 
 
 
 y1

In [213]:
plot(t, q)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,3.93701 43.4296,3.93701 48.9559,4.86704 54.4822,6.72711 60.0086,9.51721 65.5349,13.2373 71.0612,17.8875 76.5876,23.4677 82.1139,29.9779 87.6402,37.4182 
 93.1666,45.7885 98.6929,55.0888 104.219,65.3192 109.746,76.4796 115.272,88.57 120.798,101.591 126.325,115.541 131.851,130.422 137.377,146.232 142.904,162.973 
 148.43,180.643 153.956,199.244 159.483,217.168 165.009,234.162 170.535,250.225 176.062,265.359 181.588,279.563 187.114,292.837 192.641,305.18 198.167,316.594 
 203.693,327.078 209.22,336.631 214.746,345.255 220.272,352.948 225.799,359.712 231.325,365.545 236.851,370.448 242.378,374.422 247.904,377.465 253.43,379.578 
 258.957,380.762 264.483,381.015 270.009,380.338 275.536,378.731 281.062,376.194 286.588,372.727 292.115,368.331 297.641,363.004 303.167,356.747 308.694,349.559 
 314.22,342.002 319.746,334.836 325.273,328.564 330.799,323.221 336.325,318.809 341.852,315.326 347.378,312.774 352.904,311.151 358.431,310.343 363.957,310.125 
 369.483,310.303 375.01,310.722 380.536,311.259 386.062,311.828 391.589,312.37 397.115,312.849 402.641,313.248 408.168,313.563 413.694,313.797 419.22,313.959 
 424.747,314.063 430.273,314.12 435.799,314.142 441.326,314.14 446.852,314.123 452.378,314.097 457.905,314.068 463.431,314.04 468.957,314.014 474.484,313.992 
 480.01,313.974 485.536,313.96 491.063,313.951 496.589,313.944 502.115,313.94 507.642,313.938 513.168,313.938 518.694,313.939 524.221,313.94 529.747,313.941 
 535.273,313.943 540.8,313.944 546.326,313.945 551.852,313.946 557.379,313.947 562.905,313.948 568.431,313.948 573.958,313.948 579.484,313.949 585.01,313.949 
 590.537,313.949 596.063,313.949 
 "/>
 
 
 
 
 y1

In [214]:
plot(t, v)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,112.885 43.4296,126.292 48.9559,139.698 54.4822,153.104 60.0086,166.511 65.5349,179.917 71.0612,193.324 76.5876,206.73 82.1139,220.137 87.6402,233.543 
 93.1666,246.95 98.6929,260.356 104.219,273.763 109.746,287.169 115.272,300.576 120.798,313.982 126.325,327.389 131.851,340.795 137.377,354.202 142.904,367.608 
 148.43,381.015 153.956,371.259 159.483,357.852 165.009,344.446 170.535,331.039 176.062,317.633 181.588,304.226 187.114,290.82 192.641,277.413 198.167,264.007 
 203.693,250.6 209.22,237.194 214.746,223.787 220.272,210.381 225.799,196.974 231.325,183.568 236.851,170.161 242.378,156.755 247.904,143.348 253.43,129.942 
 258.957,116.535 264.483,103.129 270.009,89.7223 275.536,76.3158 281.062,62.9093 286.588,49.5028 292.115,36.0963 297.641,22.6898 303.167,9.28335 308.694,3.93701 
 314.22,9.59725 319.746,22.4651 325.273,35.8716 330.799,49.2781 336.325,62.6846 341.852,76.0911 347.378,89.4975 352.904,101.236 358.431,109.739 363.957,115.459 
 369.483,118.915 375.01,120.631 380.536,121.086 386.062,120.695 391.589,119.793 397.115,118.639 402.641,117.418 408.168,116.257 413.694,115.231 419.22,114.377 
 424.747,113.705 430.273,113.205 435.799,112.857 441.326,112.637 446.852,112.516 452.378,112.469 457.905,112.474 463.431,112.511 468.957,112.566 474.484,112.628 
 480.01,112.689 485.536,112.745 491.063,112.792 496.589,112.83 502.115,112.859 507.642,112.88 513.168,112.894 518.694,112.902 524.221,112.906 529.747,112.906 
 535.273,112.905 540.8,112.903 546.326,112.899 551.852,112.896 557.379,112.893 562.905,112.891 568.431,112.889 573.958,112.887 579.484,112.886 585.01,112.885 
 590.537,112.884 596.063,112.884 
 "/>
 
 
 
 
 y1

In [225]:
root = RegionTrees.AdaptiveSampling(mpc.MPCRefinery(), SVector(-9, -9), SVector(19, 19))

Cell: RegionTrees.HyperRectangle{2,Float64}([-9.0,-9.0],[19.0,19.0])

In [226]:
plt = plot(xlim=(-10, 10), ylim=(-10, 10), legend=nothing)
queue = [root]
i = 0
while !isempty(queue) 
    i += 1
    cell = pop!(queue)
    v = hcat(collect(RegionTrees.vertices(cell.boundary))...)
    plot!(v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]])
    if !RegionTrees.isleaf(cell)
        append!(queue, RegionTrees.children(cell))
    end
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 304.335,211.035 312

In [228]:
approx_controller = (t, q, v) -> begin
    x = [q, v]
    leaf = RegionTrees.findleaf(root, x)
    u = mpc.evaluate(leaf, x)
    u[1]
end

(::#43) (generic function with 1 method)

In [229]:
t, q, v = simulate(approx_controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  0.0016498,0.00150141,0.00128105,0.00103288,0.000787533,0.000564449,0.000374243,0.000220987,0.000104216,2.05918e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.00148396,-0.00220356,-0.00248174,-0.00245345,-0.00223084,-0.00190206,-0.00153256,-0.00116771,-0.000836245,-0.000553835])

In [230]:
plot(q, v, xlim=(-10, 10), ylim=(-10, 10))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,192.476 596.063,198.132 595.221,203.788 593.536,209.444 591.009,215.101 587.64,220.757 583.429,226.413 578.375,232.069 572.479,237.725 565.741,243.381 
 558.16,249.038 549.737,254.694 540.472,260.35 530.365,266.006 519.415,271.662 507.623,277.318 494.989,282.975 481.512,288.631 467.194,294.287 452.033,299.943 
 436.029,305.599 419.183,303.05 402.717,300.273 386.665,295.611 371.307,292.198 356.457,288.442 342.166,284.196 328.507,279.305 315.577,273.649 303.49,267.993 
 292.244,262.337 281.841,256.681 272.28,251.024 263.561,245.368 255.685,239.712 248.651,234.056 242.459,228.4 237.109,222.744 232.602,217.087 228.937,211.431 
 226.114,205.775 224.134,200.119 222.996,194.463 222.7,188.806 223.246,183.15 224.635,177.494 226.866,171.838 229.939,166.182 233.855,160.526 238.613,154.869 
 244.213,149.213 250.655,143.557 257.94,137.901 266.067,133.308 274.878,134.887 283.454,139.224 291.384,144.516 298.526,150.037 304.845,155.589 310.338,160.626 
 315.081,165.947 319.032,171.603 322.14,177.259 324.406,182.915 325.83,188.571 326.411,193.906 326.198,198.998 325.227,201.677 323.857,203.061 322.281,203.372 
 320.658,202.847 319.114,201.724 317.737,200.223 316.583,198.536 315.68,196.824 315.033,195.21 314.626,193.782 314.431,192.827 314.379,192.185 314.422,191.797 
 314.523,191.605 314.653,191.554 314.791,191.598 314.921,191.7 315.037,191.829 315.133,191.967 315.209,192.097 315.265,192.212 315.305,192.308 315.33,192.384 
 315.343,192.44 315.349,192.479 315.348,192.504 315.344,192.517 315.338,192.523 315.331,192.522 315.324,192.518 315.318,192.512 315.312,192.505 315.308,192.498 
 315.305,192.492 315.302,192.486 
 "/>
 
 
 
 
 y1